# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [48]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json

In [16]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = ""

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
#api_key = os.getenv('CLAUDE_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    if api_key.startswith("sk-ant-"):
        print("An API key was found!  Looks like a CLAUDE key.  Looks good so far!")
    else:
        print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()

In [29]:
# here is the question; type over this to ask something new

question = """
Where on earth can you pan for gold and get the best results?
"""

system_prompt = question

user_prompt = "Please respond with the top 10 places sorted by decending distance from Las Vegas, NV.  Respond in markdown."


In [28]:
# Get gpt-4o-mini to answer, with streaming
stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    stream = True
    )
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Here are the top 10 places where you can pan for gold, sorted by distance from Las Vegas, NV:

| Rank | Location                     | Distance from Las Vegas |
|------|------------------------------|-------------------------|
| 1    | Goldfield, NV                | 188 miles               |
| 2    | Pahrump, NV                  | 66 miles                |
| 3    | Beatty, NV                   | 113 miles               |
| 4    | Rhyolite, NV                 | 120 miles               |
| 5    | Death Valley National Park, CA| 130 miles               |
| 6    | Walker Lake, NV              | 217 miles               |
| 7    | Tonopah, NV                  | 218 miles               |
| 8    | Ely, NV                      | 300 miles               |
| 9    | Prescott, AZ                 | 270 miles               |
| 10   | San Bernardino Mountains, CA  | 240 miles               |

These locations offer opportunities for gold panning and might yield some interesting finds. Always check local regulations and land access conditions before heading out!

In [55]:
payload = {
        "messages": [{"role": "user", "content": system_prompt + ". " + user_prompt}],
        "stream": True,
        "max_tokens": 2000,
        "model": MODEL_LLAMA
    }

In [66]:
# Get Llama 3.2 to answer
# response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
fullresponse = ""
with requests.post(OLLAMA_API, json=payload, stream=True) as response:
        for line in response.iter_lines():
            if line:
                # Parse the JSON from the line bytes directly
                chunk = json.loads(line.decode('utf-8'))
                fullresponse += chunk["message"]["content"]
                print(chunk["message"]["content"], end="", flush=True)
                
# display(Markdown(response.json()['message']['content']))
# response = ""
# display_handle = display(Markdown(""), display_id=True)
# for chunk in stream:
#     response += chunk.choices[0].delta.content or ''
#     response = response.replace("```","").replace("markdown", "")
#     update_display(Markdown(response), display_id=display_handle.display_id)

# Top 10 Places to Pan for Gold in the US (Sorted by Distance from Las Vegas, NV)

Here are the top 10 places to pan for gold in the United States, listed in descending order by distance from Las Vegas, NV:

### 1. **Carson River, California** - 160 miles (257 km) from Las Vegas
The Carson River is a popular spot for gold panning, with many recreational gold prospectors and miners searching for nuggets.

### 2. **American River, California** - 170 miles (274 km) from Las Vegas
Located in the Sierra Nevada mountains, the American River offers plenty of opportunities to find gold using pan and sluice boxes.

### 3. **Forks of the American River, California** - 175 miles (282 km) from Las Vegas
This spot is known for its rich history of gold mining, with many famous prospectors searching for nuggets in the area.

### 4. **South Platte River, Colorado** - 560 miles (901 km) from Las Vegas
The South Platte River offers a unique opportunity to pan for gold in the heart of the Rocky Mountains